In [143]:
import os
import torch
import numpy as np
from tqdm import tqdm
from time import time
import pandas as pd
import torch_choice
from torch_choice.data import ChoiceDataset
from sklearn.preprocessing import LabelEncoder

In [144]:
torch.__version__

'1.13.0+cu117'

In [145]:
INPUT_PATH = "/home/tianyudu/Data/supermarket_simulation/1"
OUTPUT_PATH = "./benchmark_data"

In [146]:
user_latents = pd.read_csv(os.path.join(INPUT_PATH, "user_latents.csv")).drop(columns="price_coef")
item_latents = pd.read_csv(os.path.join(INPUT_PATH, "item_latents.csv"))
item_sess_price = pd.read_csv(os.path.join(INPUT_PATH, "item_sess_price.tsv"), sep="\t", header=None, names=["item_id", "session_id", "price"])

In [147]:
print(item_sess_price.columns)

Index(['item_id', 'session_id', 'price'], dtype='object')


In [148]:
for i in range(30):
    user_latents.rename(columns={str(i): f"user_latent_{i}"}, inplace=True)
    item_latents.rename(columns={str(i): f"item_latent_{i}"}, inplace=True)

In [149]:
user_latents

,user_latent_0,user_latent_1,user_latent_2,user_latent_3,user_latent_4,user_latent_5,user_latent_6,user_latent_7,user_latent_8,user_latent_9,...,user_latent_21,user_latent_22,user_latent_23,user_latent_24,user_latent_25,user_latent_26,user_latent_27,user_latent_28,user_latent_29,user_id
0,1.652043,0.896975,-0.320634,-0.725527,0.408554,0.411511,0.260232,0.794857,-1.238917,-2.424044,...,-1.489928,-0.541257,-1.069119,-1.686813,-0.793135,-0.410285,-1.002112,1.769302,1.412756,10
1,-1.424878,-2.330930,-0.484043,1.228705,-1.149234,-0.656144,0.494841,-0.267629,-0.998971,-0.284150,...,-1.568228,-0.382963,0.241070,-0.866551,-0.399027,0.903710,0.464493,-0.947236,-0.819285,11
2,0.791819,-0.233433,1.250640,-0.016727,-0.755141,-0.333004,-1.005224,-0.929233,0.078521,-0.061093,...,0.546483,0.407443,0.900532,-0.420322,-0.665277,-0.658553,0.750101,1.715973,1.043971,12
3,-1.537094,1.109955,0.071588,-0.412510,-2.458540,-0.297457,0.038190,-0.793976,-0.382170,0.908736,...,-1.070785,-0.097273,-0.342732,-0.276352,0.980518,0.095073,-0.339366,1.464080,0.130827,13
4,-0.071070,-0.201505,-0.112519,-0.736719,-0.648391,-0.477821,-0.313232,0.583525,-1.649237,-1.826413,...,0.335774,-0.276055,-0.588549,1.092662,1.165659,-0.252770,-0.264086,1.595825,1.266879,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-0.629963,-0.936433,-0.609025,0.894321,-1.458818,1.589152,-0.119949,-0.043622,-0.838796,-0.314547,...,1.256244,-0.543903,1.063730,1.437808,0.594138,-1.140934,0.504068,-1.268392,0.215705,1495
496,-0.654049,0.349450,1.117007,1.476670,0.562758,-1.577097,-1.557933,-1.695043,-1.035915,1.517097,...,0.215489,-0.569829,-1.409151,2.635519,1.012705,-0.224135,1.259907,1.344729,-1.647557,1496
497,0.292110,0.667330,-2.595917,0.668047,1.103777,0.440590,0.118690,-0.583405,0.945426,-0.013941,...,0.810359,-0.906858,0.154597,-0.389211,-0.316069,1.118300,-0.643973,-1.474082,-1.031294,1497
498,-0.316736,-0.164831,0.154419,-0.060956,-0.383504,-0.004693,1.297727,-0.747652,-0.574622,-0.405096,...,1.486630,-0.977136,1.796335,1.412003,0.516029,-0.759676,1.773111,1.008517,-0.648832,1498


In [150]:
item_latents

,item_latent_0,item_latent_1,item_latent_2,item_latent_3,item_latent_4,item_latent_5,item_latent_6,item_latent_7,item_latent_8,item_latent_9,...,item_latent_21,item_latent_22,item_latent_23,item_latent_24,item_latent_25,item_latent_26,item_latent_27,item_latent_28,item_latent_29,item_id
0,-1.180416,0.780843,-0.609615,-0.009930,-0.346506,0.256034,-1.544774,0.894837,-0.260633,-0.108083,...,-0.078928,1.246908,-1.235101,0.919382,-0.386309,-1.631385,1.979308,0.631791,-0.231759,0
1,0.076907,-0.546282,-0.066576,-0.222129,0.767167,0.454014,1.033211,0.681780,-0.436766,1.428369,...,1.879548,0.396369,-0.077428,-0.268600,0.047633,-1.203710,0.728284,-0.449082,-0.634305,1
2,1.194630,0.956158,-0.116020,1.457069,0.475462,0.462169,0.511546,-0.962028,-0.687628,0.376553,...,-1.500292,0.294344,-1.508859,-0.205371,-0.005506,-0.356446,-0.311747,0.353643,2.078901,2
3,-1.047516,0.768583,-1.523498,0.106635,0.837008,0.020559,1.783410,0.153145,-0.002027,0.354408,...,-0.561901,1.048928,-2.547275,-0.059737,-0.216024,0.420048,1.147283,-0.968257,-1.609774,3
4,0.355223,1.116302,1.122433,-1.107433,1.780376,0.707121,0.287654,-0.955454,2.414806,1.184566,...,0.699091,-0.455226,-0.620574,-1.204995,-0.153397,0.265757,-0.636522,-1.009489,1.319974,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-1.030201,-1.918512,1.597233,0.344906,-0.548310,-0.795283,0.502118,-0.843086,-0.221221,0.934821,...,1.873550,0.430558,0.022867,0.600208,0.283746,0.684030,0.944583,0.381152,-0.629110,495
496,0.836681,-0.903445,-0.931781,-0.641818,-0.698298,1.665656,1.200952,0.039846,-0.758340,-0.431396,...,-1.064213,1.223528,1.208416,-0.506720,1.334080,0.845463,0.889762,0.307895,-0.591260,496
497,-0.269530,0.383244,-0.669919,1.053787,-0.408319,0.331533,-0.818428,-0.973172,0.687965,-0.280867,...,1.064936,0.797903,-0.264837,1.365297,0.003599,0.313854,-0.841400,0.863058,-1.192473,497
498,0.765047,-0.008431,-0.340475,0.110560,0.695212,1.938753,0.075895,0.022364,-1.360376,0.955210,...,1.155430,-1.438004,-0.755103,-0.381443,-0.026872,-0.480775,-1.370354,0.072734,-0.082506,498


In [151]:
train_all = pd.read_csv(os.path.join(INPUT_PATH, "train_all.tsv"), sep="\t", header=None, names=["user_id", "item_id", "session_id", "3"])

In [152]:
print("Median number of item appearance:", train_all.value_counts("item_id").mean())
print("Median number of user appearance:", train_all.value_counts("user_id").mean())

Median number of item appearance: 3500.2
Median number of user appearance: 3500.2


## Subsample Data for Benchmarking

In [163]:
ALL_ITEMS = item_latents["item_id"].unique()
ALL_USERS = user_latents["user_id"].unique()
TOTAL_NUM_RECORDS = len(train_all)
# subsample amount.
NUM_USERS = 500
NUM_RECORDS = 100000
NUM_LATENTS = 10
if NUM_LATENTS is not None:
    user_latents.drop(columns=[f"user_latent_{i}" for i in range(NUM_LATENTS, 30)], inplace=True)
    item_latents.drop(columns=[f"item_latent_{i}" for i in range(NUM_LATENTS, 30)], inplace=True)

In [164]:
# for NUM_ITEMS in [10, 20, 30, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]:
tag = "num_records_experiment"
for NUM_ITEMS in [30]:
    # randomly choose items and users participating into the subset for performance benchmark.
    selected_items = np.random.choice(ALL_ITEMS, size=NUM_ITEMS, replace=False)
    selected_users = np.random.choice(ALL_USERS, size=NUM_USERS, replace=False)

    suitable_records = train_all[train_all["item_id"].isin(selected_items) & train_all["user_id"].isin(selected_users)]
    print(f"{len(suitable_records)} records are suitable for simulation.")
    if NUM_RECORDS is None:
        records = suitable_records
        NUM_RECORDS = len(records)
    else:
        records = suitable_records.sample(NUM_RECORDS, replace=False)

    print("Median number of item appearance:", records.value_counts("item_id").median())
    print("Median number of user appearance:", records.value_counts("user_id").median())
    # retrieve the corresponding latent vectors of selected users and items.
    relevant_user_latents = user_latents[user_latents["user_id"].isin(selected_users)]
    relevant_item_latents = item_latents[item_latents["item_id"].isin(selected_items)]
    item_encoder = LabelEncoder().fit(selected_items)
    user_encoder = LabelEncoder().fit(selected_users)

    # encode the item and user ids.
    relevant_item_latents["item_id"] = item_encoder.transform(relevant_item_latents["item_id"].values)
    relevant_user_latents["user_id"] = user_encoder.transform(relevant_user_latents["user_id"].values)
    records["item_id"] = item_encoder.transform(records["item_id"].values)
    records["user_id"] = user_encoder.transform(records["user_id"].values)
    # sort the item latents by item id.
    relevant_item_latents = relevant_item_latents.set_index("item_id").loc[np.arange(NUM_ITEMS)]
    # sort the user latents by user id.
    relevant_user_latents = relevant_user_latents.set_index("user_id").loc[np.arange(NUM_USERS)]
    dataset = ChoiceDataset(item_index=torch.LongTensor(records["item_id"].values),
                            user_index=torch.LongTensor(records["user_id"].values),
                            user_latents=torch.FloatTensor(relevant_user_latents.values),
                            item_latents=torch.FloatTensor(relevant_item_latents.values),
                            num_items=NUM_ITEMS)

    torch.save(dataset, os.path.join(OUTPUT_PATH, f"simulated_choice_data_{tag}.pt"))

108664 records are suitable for simulation.
Median number of item appearance: 3153.0
Median number of user appearance: 186.5
No `session_index` is provided, assume each choice instance is in its own session.


/tmp/ipykernel_5818/604184842.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_item_latents["item_id"] = item_encoder.transform(relevant_item_latents["item_id"].values)


In [165]:
dataset

ChoiceDataset(label=[], item_index=[100000], provided_num_items=[1], user_index=[100000], session_index=[100000], item_availability=[], user_latents=[500, 10], item_latents=[30, 10], device=cpu)

In [166]:
# tag = "full"
# torch.save(dataset, os.path.join(OUTPUT_PATH, f"simulated_choice_data_{tag}.pt"))

## Pickle the Dataset

## Prepare Dataset for `mlogit` Model

In [167]:
df_record_list = list()
for record_id in tqdm(range(NUM_RECORDS)):
    item_chosen = records.iloc[record_id]["item_id"]
    one_hot_item_chosen = np.zeros(NUM_ITEMS)
    one_hot_item_chosen[item_chosen] = 1
    # dataframe for this single record.
    df_single_record = pd.DataFrame({
        "session_id": record_id,
        "item_id": np.arange(NUM_ITEMS),
        "choice": one_hot_item_chosen,
        "user_id": records.iloc[record_id]["user_id"]})
    df_record_list.append(df_single_record)

100%|██████████| 100000/100000 [00:21<00:00, 4550.71it/s]


In [168]:
df_records = pd.concat(df_record_list, axis=0)

In [169]:
df_records

,session_id,item_id,choice,user_id
0,0,0,0.0,29
1,0,1,0.0,29
2,0,2,0.0,29
3,0,3,0.0,29
4,0,4,1.0,29
...,...,...,...,...
25,99999,25,0.0,199
26,99999,26,0.0,199
27,99999,27,0.0,199
28,99999,28,0.0,199


In [170]:
df_records = df_records.merge(relevant_user_latents, on="user_id", how="left")
df_records = df_records.merge(relevant_item_latents, on="item_id", how="left")

In [171]:
df_records.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000000 entries, 0 to 2999999
Data columns (total 24 columns):
 #   Column         Dtype  
---  ------         -----  
 0   session_id     int64  
 1   item_id        int64  
 2   choice         float64
 3   user_id        int64  
 4   user_latent_0  float64
 5   user_latent_1  float64
 6   user_latent_2  float64
 7   user_latent_3  float64
 8   user_latent_4  float64
 9   user_latent_5  float64
 10  user_latent_6  float64
 11  user_latent_7  float64
 12  user_latent_8  float64
 13  user_latent_9  float64
 14  item_latent_0  float64
 15  item_latent_1  float64
 16  item_latent_2  float64
 17  item_latent_3  float64
 18  item_latent_4  float64
 19  item_latent_5  float64
 20  item_latent_6  float64
 21  item_latent_7  float64
 22  item_latent_8  float64
 23  item_latent_9  float64
dtypes: float64(21), int64(3)
memory usage: 572.2 MB


In [172]:
print(tag)

num_records_experiment


In [173]:
df_records.to_csv(os.path.join(OUTPUT_PATH, f"simulated_choice_data_{tag}.csv"), index=False)